In [28]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
import scipy.ndimage as ndimage
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [9]:
capture = cv2.VideoCapture('data/video.mov')

n = 0
frames = []
while True:
    successful, next_frame = capture.read()
    if not successful:
        # No more frames to read
        print("Processed %d frames" % n)
        break
    frames.append(next_frame)
    n += 1
# Now we have an image! We can process that as we would.

# We have to give up the file at the end.
capture.release()

Processed 101 frames


In [11]:
height, width, channels = frames[0].shape
height, width, channels

print(frames[0].shape)

print(type(frames[0]))


(1920, 1440, 3)
<class 'numpy.ndarray'>


In [13]:
def downsize_frame(frame, factor = 2, sigma = 9):
  frame_blurred = cv2.GaussianBlur(frame, (sigma, sigma), 0)
  return frame_blurred[::factor, ::factor]

factor = 4
downsized_size = (width // factor, height // factor) # w,h
output_path = 'data/output_downsized.mp4'
output_format = cv2.VideoWriter_fourcc('M','P','4','V')
output_fps = 30
downsized_output = cv2.VideoWriter(output_path, output_format, output_fps, downsized_size)
downsized_frames = []

for frame in frames:
  output_frame = downsize_frame(frame, factor=factor)
  downsized_frames.append(output_frame)
  downsized_output.write(output_frame)

downsized_output.release()

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [14]:
# first model
# a bunch of 3x3 convolutions
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

# test = cv2.resize(frames[0], (10, 10))
# test_r = torch.tensor(test[:, :, 0]) # red component
test = torch.from_numpy(np.array(downsized_frames[:10])).float().to(device)

print(test.shape)

print("hi")

Using cpu device
torch.Size([10, 480, 360, 3])
hi


In [16]:
print(test.size())

torch.Size([10, 480, 360, 3])


In [17]:
shaped_test = torch.einsum('ijkl -> lijk', test)
# in_channels, batch size, w, h print(shaped_test.shape)

In [18]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv3d(3, 3, (3, 3, 3), bias=True, padding=1)
        
#         # self.conv2 = nn.Conv2d(3, 3, 3)
#         # add more later?

#     def forward(self, x):
#         identity = x.clone()
#         upsampled = nn.Upsample(scale_factor = 2, mode = 'bicubic')(identity)
#         out = F.relu(self.conv1(x))
#         out += upsampled
#         # out = F.relu(self.conv2(out))
        # rneturn out
net = nn.Sequential(
    nn.Conv3d(3, 16, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(16, 32, (3, 1, 1), padding=(1, 0, 0), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 32, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 32, (3, 1, 1), padding=(1, 0, 0), device=device),
    nn.ReLU(),
    nn.Upsample(scale_factor = 4, mode = 'bicubic'),
    nn.Conv3d(32, 32, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 32, (3, 1, 1), padding=(1, 0, 0), device=device),
    nn.ReLU(),
    nn.Conv3d(32, 16, (1, 3, 3), padding=(0, 1, 1), device=device),
    nn.ReLU(),
    nn.Conv3d(16, 3, (3, 1, 1), padding=(1, 0, 0), device=device)
)
torch.manual_seed(0)
print(shaped_test.size())

# a = test.reshape(test, (1, ...))
# print(test_res)


torch.Size([3, 10, 480, 360])


In [19]:
print(shaped_test.size())



torch.Size([3, 10, 480, 360])


In [20]:
print(shaped_test.size())
print(shaped_test[0])

net_test_result = net(shaped_test)
print(net_test_result)
print(net_test_result.shape)

torch.Size([3, 10, 480, 360])
tensor([[[ 61.,  63.,  63.,  ...,  12.,  13.,  13.],
         [ 63.,  64.,  64.,  ...,  13.,  13.,  12.],
         [ 63.,  64.,  64.,  ...,  13.,  13.,  13.],
         ...,
         [ 35.,  35.,  34.,  ...,  67., 112., 119.],
         [ 35.,  35.,  34.,  ...,  44.,  79., 116.],
         [ 35.,  35.,  33.,  ...,  38.,  50.,  90.]],

        [[ 64.,  60.,  58.,  ...,  12.,  13.,  12.],
         [ 65.,  63.,  59.,  ...,  13.,  13.,  12.],
         [ 65.,  64.,  61.,  ...,  13.,  13.,  13.],
         ...,
         [ 34.,  34.,  34.,  ...,  78., 116., 119.],
         [ 34.,  34.,  34.,  ...,  47.,  90., 119.],
         [ 34.,  34.,  33.,  ...,  40.,  56., 101.]],

        [[ 62.,  63.,  62.,  ...,  12.,  12.,  11.],
         [ 63.,  63.,  63.,  ...,  12.,  12.,  12.],
         [ 64.,  64.,  63.,  ...,  12.,  13.,  12.],
         ...,
         [ 33.,  33.,  33.,  ...,  90., 119., 118.],
         [ 33.,  33.,  33.,  ...,  52., 100., 119.],
         [ 33.,  33.,  

In [21]:
print(net)

Sequential(
  (0): Conv3d(3, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (1): ReLU()
  (2): Conv3d(16, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
  (3): ReLU()
  (4): Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (5): ReLU()
  (6): Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
  (7): ReLU()
  (8): Upsample(scale_factor=4.0, mode='bicubic')
  (9): Conv3d(32, 32, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (10): ReLU()
  (11): Conv3d(32, 32, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
  (12): ReLU()
  (13): Conv3d(32, 16, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  (14): ReLU()
  (15): Conv3d(16, 3, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0))
)


In [21]:
test_frame = test[0] # <class 'torch.Tensor'>
test_frame = torch.einsum('ijk -> kij', test_frame) # torch.Size([3, 480, 360])
print(test_frame.shape)

torch.Size([3, 480, 360])


In [26]:
pixel_shuffle = nn.PixelShuffle(3)
input = torch.randn(9, 4, 4) # r^2 = 9, w = 4, h = 4
output = pixel_shuffle(input)
print(output.size())

print(input)
print(output)


torch.Size([1, 12, 12])
tensor([[[ 8.3380e-01,  1.2700e+00,  1.7549e-01, -2.4588e-01],
         [ 1.5756e+00,  9.3345e-01,  4.7775e-01,  9.9757e-01],
         [-1.8312e-01, -6.6178e-02,  4.1474e-01, -1.3449e+00],
         [-1.1122e+00, -3.2281e-01,  5.3041e-01, -5.1192e-01]],

        [[ 3.2130e-01, -1.0768e+00,  6.2895e-01,  1.8093e-01],
         [ 8.3599e-02, -6.5008e-01,  5.9877e-01, -9.3046e-01],
         [-7.0778e-01, -3.6393e-01, -1.6026e-03,  2.6617e-02],
         [ 1.1753e-01,  1.8722e-01,  9.5401e-01,  2.6710e+00]],

        [[ 1.4114e+00,  7.7821e-01, -9.7807e-01,  3.1295e-01],
         [ 3.1307e-01,  1.5526e-01,  8.0901e-01, -1.1968e+00],
         [ 5.8138e-01, -1.4382e+00,  5.8315e-01,  6.4055e-01],
         [-9.4375e-02, -3.0579e-01,  1.5090e+00,  9.0511e-01]],

        [[-1.6416e+00, -9.2762e-01, -6.4997e-01,  2.3729e-01],
         [-1.1454e+00, -5.7225e-01,  7.8862e-01, -1.6651e+00],
         [ 7.0899e-01,  6.1408e-01, -6.9340e-01,  1.3284e-01],
         [ 1.9332e-01, -3

In [56]:
a = [1,2,3]
b = [*a]
print(b)

b = torch.tensor([[1,2,3]])
c = torch.tensor([[1,2,3],
                  [4,5,6]])

print(np.concatenate((b, c), axis=0))

print(list(i for i in a))

print(torch.rand(2,2))

[1, 2, 3]
[[1 2 3]
 [1 2 3]
 [4 5 6]]
[1, 2, 3]
tensor([[0.8132, 0.1757],
        [0.1278, 0.8218]])


In [57]:
# attempt at doing 2016 result thing on single frame
# good walkthrough: https://medium.com/@zhuocen93/an-overview-of-espcn-an-efficient-sub-pixel-convolutional-neural-network-b76d0a6c875e

# only for one frame; doesn't include batch size TODO
# need to normalize frame before passing TODO

# model uses one color channel at a time
# 5x5 kernel, in: 1, out: 64
# 3x3 kernel, in: 64, out: 32
# 3x3 kernel, in: 32, out: r^2

# let's say we upsample by r=4 for now? <- change later
r = 4
r_squared = 16

# only for one color channel
class Model2016(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(5,5), stride=1, padding=2)
        self.act1 = nn.ReLU()
        # self.drop1 = nn.Dropout(0.3) # randomly dropout some values <- can try later
 
        self.conv2 = nn.Conv2d(64, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        # self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        # self.flat = nn.Flatten()
        self.conv3 = nn.Conv2d(32, r_squared, kernel_size=(3,3), stride=1, padding=1)
        self.act3 = nn.ReLU()
 
    def forward(self, x):
        # input 1x?x?, output 64x?x?
        x = self.act1(self.conv1(x))
        # input 64x?x?, output 32x?x?
        x = self.act2(self.conv2(x))
        # input 32x?x?, output r^2x?x?
        x = self.act3(self.conv3(x))
        return x

### stuff to feed into trainer
model = Model2016()
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
pixel_shuffle = nn.PixelShuffle(r)
### 

HR_shape = (3, test_frame.shape[1]*r, test_frame.shape[2]*r) # High-Res shape
ground_truth_frames = [torch.rand(*HR_shape)] # TODO replace with real ground truth
train_frames = [test_frame] # TODO replace with actual frames

n_epochs = 20
for epoch in range(n_epochs):
    for i, (frame, truth) in enumerate(zip(train_frames, ground_truth_frames)):

        for channel_frame in frame:
            channel_frame = channel_frame[np.newaxis, :, :]
            temp = model(channel_frame)
            deconv = pixel_shuffle(temp)
        
        deconvolved = torch.cat(list(pixel_shuffle(model(channel_frame[np.newaxis, :, :]))
                                    for channel_frame in frame), axis=0) # deconvolved 3 x r*? x r*?
        
        loss = loss_fn(deconvolved, truth)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))


torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<class 'torch.Tensor'>
torch.Size([3, 1920, 1440])
<cl

(1, 1920, 1440)
